# Lesson 3: Introduction to model parameter optimization

## Lesson goals:

1. Discuss challenges from Lesson 2:
 - Measure and plot an "f-I" curve
 - Manually explore model parameters controlling input resistance and "rheobase"
2. Introduction to optimization with scipy.minimize
3. Use scipy.minimize to optimize model input resistance

## Challenge for next week:
4. Devise an objective function for optimization of rheobase.

## Papers for next week:
5. Papers focusing on filtering of synaptic signals from dendrites to soma

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
h.load_file('stdrun.hoc')

## 1. Discuss challenges from Lesson 2:
### Measure and plot an "f-I" curve
First, let's build our Hodgkin-Huxley squid axon compartment:

In [ ]:
soma = h.Section()

soma.insert('hh')

t = h.Vector()
soma_voltage = h.Vector()
t.record(h._ref_t)  # record the time stamp
soma_voltage.record(soma(0.5)._ref_v)  # record the voltage across the membrane in a segment

step_current_stim = h.IClamp(soma(0.5))
step_current_stim.amp = 10.  # amplitude in nanoAmps
step_current_stim.dur = 200.  # duration in milliseconds
step_current_stim.delay = 200.  # start time of current injection

step_current_rec = h.Vector()
step_current_rec.record(step_current_stim._ref_i)

h.tstop = 600.

Let's store the initial values of each of the ion channel conductances:

In [ ]:
gl_init = soma(0.5).hh.gl
gnabar_init = soma(0.5).hh.gnabar
gkbar_init = soma(0.5).hh.gkbar

print('Initial leak condutance is %.5f S/cm^2' % gl_init)
print('Initial Na condutance is %.5f S/cm^2' % gnabar_init)
print('Initial K condutance is %.5f S/cm^2' % gkbar_init)

Now, let's run a simulation that contains spikes:

In [ ]:
h.run()
plt.figure()
plt.plot(t, soma_voltage)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (mV)')

Now let's write a function to compute the times that spikes occur:

In [ ]:
def get_spike_times(t, vm, vm_threshold=0.):
    spike_times = []
    i = 0
    while i < len(t):
        if vm[i] >= vm_threshold:
            spike_times.append(t[i])
            i += 1
            while i < len(t) and vm[i] >= vm_threshold:
                i += 1
        else:
            i += 1
    return np.array(spike_times)

In [ ]:
spike_times = get_spike_times(np.array(t), np.array(soma_voltage))
print(spike_times)

Now we can calculate a firing rate from a list of spike times:

In [ ]:
def get_firing_rate(spike_times, start, stop):
    duration = (stop - start) / 1000.  # convert to seconds
    spike_count = len(np.where((spike_times >= start) & (spike_times < stop))[0])
    return spike_count / duration  # in Hz

In [ ]:
firing_rate = get_firing_rate(spike_times, 200., 400.)
print('The firing rate is %.1f Hz' % firing_rate)

Now let's simulate a range of leak conductance values, measure input resistance, and store the results:

In [ ]:
sim_history = []
stim_amp_array = np.arange(5., 55., 5.)

for stim_amp in stim_amp_array:
    sim_record = {}
    step_current_stim.amp = stim_amp
    sim_record['stim_amp'] = step_current_stim.amp
    sim_record['description'] = 'stim_amp: %.1f nA' % stim_amp
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    spike_times = get_spike_times(sim_record['t'], sim_record['soma_voltage'])
    firing_rate = get_firing_rate(spike_times, step_current_stim.delay, step_current_stim.delay + step_current_stim.dur)
    sim_record['firing_rate'] = firing_rate
    
    sim_history.append(sim_record)

Let's plot the traces:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_voltage']
    step_current_rec_array = sim_record['step_current_rec']
    description = sim_record['description']
    axes[0].plot(t_array, step_current_rec_array, label=description)
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Current (nA)')
    axes[0].set_title('Injected current')
    axes[1].plot(t_array, soma_voltage_array, label='Firing rate: %.1f Hz' % sim_record['firing_rate'])
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('Voltage (mV)')
    axes[1].set_title('Soma membrane potential')
axes[0].legend(loc='best', frameon=False)
axes[1].legend(loc='best', frameon=False)
fig.tight_layout(w_pad=0.2)

Let's plot current vs. firing rate:

In [ ]:
firing_rate_list = []
for sim_record in sim_history:
    firing_rate_list.append(sim_record['firing_rate'])

fig, ax = plt.subplots()
ax.scatter(stim_amp_array, firing_rate_list)
ax.set_xlabel('Current (nA)')
ax.set_ylabel('Firing rate (Hz)')

## Manually explore model parameters controlling input resistance and "rheobase"

First, let's measure the input resistance for the default leak conductance:

In [ ]:
def get_input_resistance(t, vm, i, start, stop, window_dur):
    
    baseline_start_index = np.where(t >= start - window_dur)[0][0]
    baseline_end_index = np.where(t >= start)[0][0]
    equil_start_index = np.where(t >= stop - window_dur)[0][0]
    equil_end_index = np.where(t >= stop)[0][0]
    delta_vm = np.abs(np.mean(vm[equil_start_index:equil_end_index]) - np.mean(vm[baseline_start_index:baseline_end_index]))
    delta_i = np.abs(np.mean(i[equil_start_index:equil_end_index]) - np.mean(i[baseline_start_index:baseline_end_index]))
    
    # Ohms = Volts / Amps; MegaOhms = milliVolts / nanoAmps
    input_res = delta_vm / delta_i
    return input_res

In [ ]:
print('Initial leak condutance is %.5f S/cm^2' % gl_init)
soma(0.5).hh.gl = gl_init

In [ ]:
step_current_stim.amp = -1.  # amplitude in nanoAmps
step_current_stim.dur = 1000.  # duration in milliseconds
step_current_stim.delay = 200.  # start time of current injection

h.tstop = 1400.

h.run()

fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].plot(t, step_current_rec, c='r')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Current (nA)')
axes[0].set_title('Injected current')
axes[1].plot(t, soma_voltage, c='k')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Voltage (mV)')
axes[1].set_title('Soma membrane potential')

fig.tight_layout()

input_resistance = get_input_resistance(np.array(t), np.array(soma_voltage), np.array(step_current_rec), start=step_current_stim.delay, 
                                        stop=step_current_stim.delay + step_current_stim.dur, window_dur=10.)
print('The input resistance is %.2f MOhm' % input_resistance)

Now let's simulate a range of leak conductance values, measure input resistance, and store the results:

Let's build an array of values to use for the parameters gl. Let's test values across multiple orders of magnitude:

In [ ]:
log10_range = 10 ** np.arange(-2., 4.)
print(log10_range)

In [ ]:
gl_param_array = gl_init * log10_range
print(gl_param_array)

In [ ]:
sim_history = []

for gl in gl_param_array:
    sim_record = {}
    sim_record['stim_amp'] = step_current_stim.amp
    sim_record['gl'] = gl
    soma(0.5).hh.gl = gl
    sim_record['description'] = 'gl: %.8f nA' % gl
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    input_resistance = get_input_resistance(sim_record['t'], sim_record['soma_voltage'], sim_record['step_current_rec'], start=200., stop=400., window_dur=10.)
    sim_record['input_resistance'] = input_resistance
    
    sim_history.append(sim_record)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_voltage']
    step_current_rec_array = sim_record['step_current_rec']
    description = sim_record['description']
    axes[0].plot(t_array, step_current_rec_array, label=description)
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Current (nA)')
    axes[0].set_title('Injected current')
    axes[1].plot(t_array, soma_voltage_array, label='R_inp: %.3f MOhm' % sim_record['input_resistance'])
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('Voltage (mV)')
    axes[1].set_title('Soma membrane potential')
axes[0].legend(loc='best', frameon=False)
axes[1].legend(loc='best', frameon=False)
fig.tight_layout(w_pad=0.9)

Let's plot conductance vs. input resistance:

In [ ]:
R_inp_val_list = []
for sim_record in sim_history:
    R_inp_val_list.append(sim_record['input_resistance'])

fig, ax = plt.subplots()
ax.scatter(gl_param_array, R_inp_val_list)
ax.set_xlabel('Conductance (S/cm^2)')
ax.set_ylabel('Input resistance (MOhm)')

Let's try plotting on a log scale:

In [ ]:
fig, ax = plt.subplots()
ax.scatter(gl_param_array, R_inp_val_list)
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Conductance (S/cm^2)')
ax.set_ylabel('Input resistance (MOhm)')

Let's use value of gl that produced an input resistance close to 5 MOhm.

In [ ]:
gl = gl_param_array[1]
soma(0.5).hh.gl = gl
print('Conductance: %.8f S/cm^2 results in Input resistance: %.1f MOhm' % (gl_param_array[1], R_inp_val_list[1]))

Now we have increased the sensitivity of the cell to injected current. 

We now expect the cell to spike in reponse to a lower amount of injected current.

"Rheobase" is that amount of current needed to cross the voltage threshold to produce a spike.

In [ ]:
step_current_stim.amp = -1.  # amplitude in nanoAmps
step_current_stim.dur = 50.  # duration in milliseconds
step_current_stim.delay = 200.  # start time of current injection

h.tstop = 300.

sim_history = []
stim_amp_array = np.arange(0.5, 5., 0.5)

for stim_amp in stim_amp_array:
    sim_record = {}
    step_current_stim.amp = stim_amp
    sim_record['stim_amp'] = step_current_stim.amp
    sim_record['description'] = 'stim_amp: %.1f nA' % stim_amp
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    spike_times = get_spike_times(sim_record['t'], sim_record['soma_voltage'])
    firing_rate = get_firing_rate(spike_times, step_current_stim.delay, step_current_stim.delay + step_current_stim.dur)
    sim_record['firing_rate'] = firing_rate
    
    sim_history.append(sim_record)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_voltage']
    step_current_rec_array = sim_record['step_current_rec']
    description = sim_record['description']
    axes[0].plot(t_array, step_current_rec_array, label=description)
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Current (nA)')
    axes[0].set_title('Injected current')
    axes[1].plot(t_array, soma_voltage_array, label='Firing rate: %.1f Hz' % sim_record['firing_rate'])
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('Voltage (mV)')
    axes[1].set_title('Soma membrane potential')
axes[0].legend(loc='best', frameon=False)
axes[1].legend(loc='best', frameon=False)
fig.tight_layout(w_pad=0.2)

It's still taking 2.5 nA of current to spike. Let's try to vary gnabar to reduce that to less than 1.0 nA.

Let's explore a range:

In [ ]:
gnabar_param_array = gnabar_init * np.arange(1., 2.75, 0.25)
print('gnabar:', gnabar_param_array)
soma(0.5).hh.gkbar = gkbar_init

In [ ]:
stim_amp = 0.5  # nA - our ambitious target for rheobase
step_current_stim.amp = stim_amp

sim_history = []

for gnabar in gnabar_param_array:
    sim_record = {}
    sim_record['stim_amp'] = step_current_stim.amp
    soma(0.5).hh.gnabar = gnabar
    sim_record['gl'] = soma(0.5).hh.gl
    sim_record['gnabar'] = soma(0.5).hh.gnabar
    sim_record['gkbar'] = soma(0.5).hh.gkbar
    
    sim_record['description'] = 'gnabar: %.3f S/cm^s' % gnabar
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    spike_times = get_spike_times(sim_record['t'], sim_record['soma_voltage'])
    firing_rate = get_firing_rate(spike_times, step_current_stim.delay, step_current_stim.delay + step_current_stim.dur)
    sim_record['firing_rate'] = firing_rate
    
    sim_history.append(sim_record)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_voltage']
    step_current_rec_array = sim_record['step_current_rec']
    description = sim_record['description']
    axes[0].plot(t_array, step_current_rec_array, label=description)
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Current (nA)')
    axes[0].set_title('Injected current')
    axes[1].plot(t_array, soma_voltage_array, label='Firing rate: %.1f Hz' % sim_record['firing_rate'])
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('Voltage (mV)')
    axes[1].set_title('Soma membrane potential')
axes[0].legend(loc='best', frameon=False)
axes[1].legend(loc='best', frameon=False)
fig.tight_layout(w_pad=0.2)

It appears that at 0.27 S/cm^2, spike threshold is crossed during the current injection.

But at 0.30 S/cm^2, spiking occurs even outside the duration of the step current!

Let's try exploring values of gkbar to compensate:

In [ ]:
soma(0.5).hh.gnabar = gnabar_param_array[-1]
gkbar_param_array = gkbar_init * np.arange(1., 1.25, 0.05)
print('gkbar:', gkbar_param_array)

In [ ]:
sim_history = []

for gkbar in gkbar_param_array:
    sim_record = {}
    sim_record['stim_amp'] = step_current_stim.amp
    soma(0.5).hh.gkbar = gkbar
    sim_record['gl'] = soma(0.5).hh.gl
    sim_record['gnabar'] = soma(0.5).hh.gnabar
    sim_record['gkbar'] = soma(0.5).hh.gkbar
    
    sim_record['description'] = 'gkbar: %.3f S/cm^s' % gkbar
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    spike_times = get_spike_times(sim_record['t'], sim_record['soma_voltage'])
    firing_rate = get_firing_rate(spike_times, step_current_stim.delay, step_current_stim.delay + step_current_stim.dur)
    sim_record['firing_rate'] = firing_rate
    
    sim_history.append(sim_record)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_voltage']
    step_current_rec_array = sim_record['step_current_rec']
    description = sim_record['description']
    axes[0].plot(t_array, step_current_rec_array, label=description)
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Current (nA)')
    axes[0].set_title('Injected current')
    axes[1].plot(t_array, soma_voltage_array, label='Firing rate: %.1f Hz' % sim_record['firing_rate'])
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('Voltage (mV)')
    axes[1].set_title('Soma membrane potential')
axes[0].legend(loc='best', frameon=False)
axes[1].legend(loc='best', frameon=False)
fig.tight_layout(w_pad=0.2)

At a value of 0.041 S/cm^2, the cell only spiked inside the duration of the step current.

### Can this manual tuning process be automated?

## 2. Introduction to optimization with scipy.minimize

Let's say we have a function that takes in free parameters, and produces a single float output:

In [ ]:
def y1(x):
    """
    The argument 'x' is an array of length 1.
    """
    y = (x[0] - 4) ** 2. + 10.
    return y

x_param_range = np.arange(-10., 10., 0.1)
y1_output_list = []
for x in x_param_range:
    y1_output = y1([x])
    y1_output_list.append(y1_output)
plt.figure()
plt.plot(x_param_range, y1_output_list)
plt.xlabel('Input parameter x')
plt.ylabel('Output objective y1')
plt.show()

Since this is a simple function, we could obtain the global minimum by graphing the function, or by solving the equation analytically. But when functions are complex, numerical optimization methods become useful tools to search for approximate local minimums.

scipy.optimize.minimize is a general interface that can use many different algorithms to search for input parameters that minimize a provided function. 

In [ ]:
from scipy.optimize import minimize
# initial guess for the input parameter
x0_array = [0.]

result = minimize(y1, x0_array, options={'disp': True})

By default, scipy.minimize uses a "gradient-based algorithm" that chooses new parameters to test based on the slope of the function around each tested point.

In [ ]:
print(result)

Well that was easy. But how do we know what parameters it tested? We'll have to manually keep track by appending the values to a global variable:

In [ ]:
x_history = []
y_history = []

def y1(x):
    """
    The argument 'x' is an array of length 1.
    """
    x_history.append(x[0])
    y = (x[0] - 4) ** 2. + 10.
    y_history.append(y)
    return y

result = minimize(y1, x0_array, options={'disp': True})

# zip is a python built-in that lets you iterate over more than one array at a time

plt.figure()
for x, y in zip(x_history, y_history):
    plt.scatter(x, y, marker='o', c='k')
plt.scatter(result.x[0], result.fun, marker='o', c='r')
plt.plot(x_param_range, y1_output_list)
plt.xlabel('Input parameter x0')
plt.ylabel('Output objective y1')
plt.show()

print(result.x, result.fun)

## 3. Use scipy.minimize to optimize model input resistance

### How can we apply this to find a value of gl that results in a target input resistance of 10 Mohm?

First we'll need a "objective function" that will return a float value that correponds to how far away a model is from its target.

For convenience, we'll have this objective error function call another function that simulates our neuron model.

We'll continue to use our stategy of saving simulation results to a global history variable.

In [ ]:
sim_history = []

def simulate_model(plot=False):
    sim_record = {}
    sim_record['stim_amp'] = step_current_stim.amp
    sim_record['gl'] = soma(0.5).hh.gl
    
    sim_record['description'] = 'gl: %.2E S/cm^s' % soma(0.5).hh.gl
    h.run()
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    input_resistance = get_input_resistance(sim_record['t'], sim_record['soma_voltage'], sim_record['step_current_rec'], 
                                            start=step_current_stim.delay, stop=step_current_stim.delay + step_current_stim.dur, 
                                            window_dur=10.)
    sim_record['input_resistance'] = input_resistance
    
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(8, 3))
        axes[0].plot(sim_record['t'], sim_record['step_current_rec'], label=sim_record['description'])
        axes[0].set_xlabel('Time (ms)')
        axes[0].set_ylabel('Current (nA)')
        axes[0].set_title('Injected current')
        axes[1].plot(sim_record['t'], sim_record['soma_voltage'], label='R_inp: %.3f MOhm' % sim_record['input_resistance'])
        axes[1].set_xlabel('Time (ms)')
        axes[1].set_ylabel('Voltage (mV)')
        axes[1].set_title('Soma membrane potential')
        axes[0].legend(loc='best', frameon=False)
        axes[1].legend(loc='best', frameon=False)
        fig.tight_layout(w_pad=0.9)
    
    return sim_record
        

def get_input_resistance_objective_error(x, target, tolerance, sim_history=None, plot=False):
    
    # expect x to be an array of length 1. The single element is a value of gl to test.
    gl = x[0]
    soma(0.5).hh.gl = gl
    
    sim_record = simulate_model(plot=plot)
    sim_record['x'] = np.copy(x)
    
    input_resistance = sim_record['input_resistance']
    error = ((target - input_resistance) / tolerance) ** 2.
    sim_record['input_resistance_error'] = error
    
    if sim_history is not None:
        sim_history.append(sim_record)
    
    return error

Let's make sure our functions work:

First, let's reset our model configuration:

In [ ]:
soma(0.5).hh.gnabar = gnabar_init
soma(0.5).hh.gkbar = gkbar_init
soma(0.5).hh.gl = gl_init

step_current_stim.amp = -1.
step_current_stim.dur = 1000.
step_current_stim.delay = 200.
h.tstop = 1400.

In [ ]:
sim_record = simulate_model(plot=True)

In [ ]:
print(sim_record)

Now, the objective function expects an array of input parameters, a target, and an error tolerance:

In [ ]:
x0 = [gl_init]
target = 10.  # Input resistance in MOhm
tolerance = 1.  # Tolerance in MOhm

error = get_input_resistance_objective_error(x0, target, tolerance)
print(error)

Now let's see if scipy.minimize can find the optimal value of gl to fit our model to the target.

In [ ]:
sim_history = []
result = minimize(get_input_resistance_objective_error, x0, options={'disp': True}, args=(target, tolerance, sim_history))

In [ ]:
print(result)

In [ ]:
x_history = []
input_resistance_history = []
error_history = []
for sim_record in sim_history:
    x_history.append(sim_record['x'][0])
    input_resistance_history.append(sim_record['input_resistance'])
    error_history.append(sim_record['input_resistance_error'])

fig, axes = plt.subplots(1, 2)
axes[0].scatter(x_history, input_resistance_history)
axes[0].set_xlabel('Leak conductance (S/cm^2)')
axes[0].set_ylabel('Input resistance (MOhm)')

axes[1].scatter(x_history, error_history)
axes[1].set_xlabel('Leak conductance (S/cm^2)')
axes[1].set_ylabel('Input resistance objective error')
fig.tight_layout()

Well that didn't work very well - this algorithm has trouble searching across multiple orders of magnitude!

Let's try another algorithm called simplex, or Nelder-Mead:

In [ ]:
sim_history = []
result = minimize(get_input_resistance_objective_error, x0, method='Nelder-Mead', options={'disp': True}, args=(target, tolerance, sim_history))

In [ ]:
print(result)

In [ ]:
x_history = []
input_resistance_history = []
error_history = []
for sim_record in sim_history:
    x_history.append(sim_record['x'][0])
    input_resistance_history.append(sim_record['input_resistance'])
    error_history.append(sim_record['input_resistance_error'])

fig, axes = plt.subplots(1, 2)
axes[0].scatter(x_history, input_resistance_history)
axes[0].set_xlabel('Leak conductance (S/cm^2)')
axes[0].set_ylabel('Input resistance (MOhm)')

axes[1].scatter(x_history, error_history)
axes[1].set_xlabel('Leak conductance (S/cm^2)')
axes[1].set_ylabel('Input resistance objective error')
fig.tight_layout()

That appears to have worked! Let's test the value of gl returned by the simplex minimization:

In [ ]:
get_input_resistance_objective_error(result.x, target, tolerance, plot=True)

## Challenge for next week:
## 4. Devise an objective function for optimization of rheobase.

We need a function that will take in a parameter array of length 2. One element will be a value of gnabar to test, and one element will be a value of gkbar to test.

The function should run simulations for a range of current injection amplitudes from 0.1 to 1.0 nA.

If spiking occurs outside the duration of the step current, a large error value should be returned.

If there are no spikes even for the largest current injection amplitude, a large error value should be returned.

Otherwise, the minimum value of current injection amplitude that led to spiking inside the duration of the step current should be compared to a target value of 0.5 nA, and return an error value based on the distance from the target.

## Papers for next week:
## 5. Papers focusing on filtering of synaptic signals from dendrites to soma

- Magee, J., Cook, E. (2000). Somatic EPSP amplitude is independent of synapse location in hippocampal pyramidal neurons Nature Neuroscience  3(9), 895-903. https://dx.doi.org/10.1038/78800
- Magee, J. Dendritic Ih normalizes temporal summation in hippocampal CA1 neurons. Nat Neurosci 2, 508–514 (1999). https://doi.org/10.1038/9158